# Import Library

In this section, we import various Python libraries that will help us perform data manipulation, visualization, preprocessing, and build machine learning regression models. Each library serves a specific purpose — from handling data to evaluating and visualizing model performance.

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
```

- **pandas (`pd`)**: Used for data manipulation and analysis. It allows easy handling of structured data such as CSV, Excel, or SQL data.
- **numpy (`np`)**: Provides mathematical functions and efficient operations on numerical arrays or matrices.
- **matplotlib.pyplot (`plt`)**: A plotting library used to create visualizations like line charts, bar charts, scatter plots, etc.
- **seaborn (`sns`)**: Built on top of Matplotlib, Seaborn provides a higher-level interface for making attractive and informative statistical graphics.

```python
from sklearn... import ...
```

- **sklearn**: This library is used for machine learning tasks such as classification, regression, clustering, and more. It provides a wide range of tools and algorithms to build and evaluate machine learning models.

```python
from xgboost import XGBRegressor
```

- **XGBRegressor**: This is an optimized version of gradient boosting that includes additional features and optimizations. It is known for its high performance and efficiency, especially on larger datasets.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import permutation_importance

from sklearn.svm import SVR
from xgboost import XGBRegressor

---

# Reading Dataset

In this part of the code, we are reading the dataset that contains the training and testing data for our machine learning model. The dataset is stored in a CSV (Comma-Separated Values) file, which is a common format for storing tabular data. We use the `pd.read_csv()` function from the `pandas` library to read the dataset into a DataFrame, which is a two-dimensional data structure that resembles a table.

In [2]:
data = pd.read_csv("data/burnout_submissions.csv")

data.describe()

,usia,jumlah_anak,usia_anak,lama_bekerja,waktu_bekerja_seminggu,beban_sks,mhs_bimbingan,work_life_balance,gaji_sesuai,1_tidak_mampu,...,4_waktu_tidak_cukup,5_tidak_berjalan_baik,6_terburu_buru,7_tidak_ada_jalan_keluar,8_masalah_menumpuk,9_ingin_menyerah,10_memikul_beban_berat,personal_vulnerability_ganjil,event_load_genap,skor_total
count,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,...,11.000000,11.0,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000,11.000000
mean,33.363636,0.818182,1.818182,6.363636,51.090909,22.538182,11.818182,3.000000,3.000000,2.090909,...,3.272727,2.0,3.181818,1.818182,2.636364,1.727273,2.545455,9.727273,14.636364,24.363636
std,3.384456,0.873863,2.088932,3.413875,6.992203,9.937572,9.526995,1.095445,1.095445,1.136182,...,1.678744,1.0,1.401298,0.981650,1.286291,1.272078,1.368476,4.649536,5.749308,9.749592
min,26.000000,0.000000,0.000000,0.000000,40.000000,10.000000,0.000000,2.000000,2.000000,1.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,5.000000,5.000000,10.000000
25%,31.500000,0.000000,0.000000,6.000000,45.500000,16.500000,5.000000,2.000000,2.000000,1.000000,...,2.000000,1.0,2.500000,1.000000,2.000000,1.000000,1.500000,6.000000,10.000000,16.000000
50%,34.000000,1.000000,2.000000,7.000000,50.000000,21.000000,9.000000,3.000000,3.000000,2.000000,...,4.000000,2.0,3.000000,2.000000,2.000000,1.000000,2.000000,11.000000,16.000000,25.000000
75%,35.500000,1.500000,3.000000,8.500000,58.000000,29.935000,17.000000,4.000000,4.000000,2.500000,...,5.000000,2.5,4.000000,2.000000,3.500000,2.000000,3.500000,12.000000,19.000000,31.500000
max,38.000000,2.000000,6.000000,10.000000,60.000000,43.000000,30.000000,5.000000,5.000000,4.000000,...,5.000000,4.0,5.000000,4.000000,5.000000,5.000000,5.000000,20.000000,22.000000,41.000000


# Define Features and Target

In this part of the code, we define which columns from our dataset will be used as **features (input variables)** and which column will be used as the **target (output variable)** for the regression model.

The goal here is to separate the data into two main parts:

* **`X` (features):** the variables that will be used by the model to make predictions.
* **`y` (target):** the variable that we want the model to predict — in this case, the **total stress score (`skor_total`)**.

In [3]:
features_to_drop = [
    'tinggal_dengan_siapa', 'kesehatan_fisik', 'kondisi_mental',
    '1_tidak_mampu', '2_kewalahan_tanggung_jawab', '3_keadaan_tidak_berpihak',
    '4_waktu_tidak_cukup', '5_tidak_berjalan_baik', '6_terburu_buru',
    '7_tidak_ada_jalan_keluar', '8_masalah_menumpuk', '9_ingin_menyerah',
    '10_memikul_beban_berat', 'personal_vulnerability_ganjil', 'PV',
    'event_load_genap', 'EV', 'skor_total', 'risiko_stres'
]

X = data.drop(columns=features_to_drop)
y = data['skor_total']

In [4]:
print(f"Number of Features: {len(X.columns)}")
print(f"Features: {X.columns.tolist()}")

Number of Features: 44
Features: ['usia', 'jenis_kelamin', 'kota_asal', 'status_pernikahan', 'jumlah_anak', 'usia_anak', 'tinggal_sendiri', 'tinggal_pasangan', 'tinggal_anak', 'tinggal_ortu', 'tinggal_mertua', 'tinggal_saudara', 'tinggal_teman', 'profesi', 'bidang', 'lama_bekerja', 'mode_bekerja', 'jarak', 'waktu_bekerja_seminggu', 'beban_sks', 'mhs_bimbingan', 'jabatan_struktural', 'jabatan_fungsional', 'sertifikasi', 'status_keaktifan', 'fisik_mata', 'fisik_punggung', 'fisik_tensi', 'fisik_lemah', 'fisik_kepala', 'fisik_obesitas', 'fisik_imun', 'fisik_carpal', 'mental_anxiety', 'mental_burnout', 'mental_depresi', 'mental_distress', 'mental_konsentrasi', 'mental_insomnia', 'mental_iritate', 'mental_lelah', 'mental_stres', 'work_life_balance', 'gaji_sesuai']


---

# Preprocessing Non-Numerical Features

Not all features in a dataset are numerical. Some columns may contain **categorical data** (e.g., gender, job position, or living situation) or **boolean values** (e.g., yes/no). Since most machine learning models require numerical input, these categorical variables need to be transformed into numbers.

In this part of the code, we identify which features are **numerical** and which are **categorical**, and then apply different preprocessing steps using **`ColumnTransformer`** — scaling numerical data and encoding categorical data.

## Step Explanation

* **`numerical_features`** → Selects all numeric columns (`int64`, `float64`) to be standardized using `StandardScaler`.
* **`categorical_features`** → Selects all non-numeric columns (`object`, `bool`) that require encoding into numbers.
* **`ColumnTransformer`** → Combines preprocessing steps for different feature types:

  * `'num'`: Applies **`StandardScaler`** to numerical columns so that all values have a mean of 0 and a standard deviation of 1.
  * `'cat'`: Applies **`OneHotEncoder`** to categorical columns, converting each category into a binary column (0 or 1).
  * `remainder='passthrough'`: Keeps any remaining columns unchanged.

## About One-Hot Encoding

**One-Hot Encoding** is a method for converting categorical data into a numerical form that machine learning algorithms can interpret. Instead of assigning arbitrary numbers (like 1, 2, 3), which would imply an order or ranking, one-hot encoding creates **binary columns** for each unique category.

This ensures that the model treats categories equally without assuming any hierarchy.

### One-Hot Encoding Explained

Suppose we have a column named `living_status` with three categories: `"Alone"`, `"With Family"`, and `"With Friends"`.

Original data:

| living_status |
| ------------- |
| Alone         |
| With Family   |
| With Friends  |
| Alone         |
| With Family   |

After applying one-hot encoding, it becomes:

| living_status_Alone | living_status_WithFamily | living_status_WithFriends |
| ------------------- | ------------------------ | ------------------------- |
| 1                   | 0                        | 0                         |
| 0                   | 1                        | 0                         |
| 0                   | 0                        | 1                         |
| 1                   | 0                        | 0                         |
| 0                   | 1                        | 0                         |

## Why This Step Matters

* Ensures **all input data is numerical**, which is required for most machine learning algorithms.
* Prevents **bias or errors** from arbitrary numeric category encoding.
* Makes model training **consistent and reliable**, especially when unseen categories appear in the test set (handled by `handle_unknown='ignore'`).

By preprocessing both numerical and categorical features properly, we ensure that the dataset is fully prepared for **model training and evaluation**.


In [6]:
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)

print(f"Numerical Features: {numerical_features}")
print(f"Categorical Features: {categorical_features}")

Numerical Features: ['usia', 'jumlah_anak', 'usia_anak', 'lama_bekerja', 'waktu_bekerja_seminggu', 'beban_sks', 'mhs_bimbingan', 'work_life_balance', 'gaji_sesuai']
Categorical Features: ['jenis_kelamin', 'kota_asal', 'status_pernikahan', 'tinggal_sendiri', 'tinggal_pasangan', 'tinggal_anak', 'tinggal_ortu', 'tinggal_mertua', 'tinggal_saudara', 'tinggal_teman', 'profesi', 'bidang', 'mode_bekerja', 'jarak', 'jabatan_struktural', 'jabatan_fungsional', 'sertifikasi', 'status_keaktifan', 'fisik_mata', 'fisik_punggung', 'fisik_tensi', 'fisik_lemah', 'fisik_kepala', 'fisik_obesitas', 'fisik_imun', 'fisik_carpal', 'mental_anxiety', 'mental_burnout', 'mental_depresi', 'mental_distress', 'mental_konsentrasi', 'mental_insomnia', 'mental_iritate', 'mental_lelah', 'mental_stres']


In [8]:
# Cek hasil preprocessing
# Pertama, kita perlu fit preprocessor dengan data
X_preprocessed = preprocessor.fit_transform(X)

# Dapatkan nama fitur setelah preprocessing
feature_names_after = preprocessor.get_feature_names_out()

print(f"Jumlah fitur sebelum preprocessing: {X.shape[1]}")
print(f"Jumlah fitur setelah preprocessing: {X_preprocessed.shape[1]}")
print(f"\nBentuk data asli: {X.shape}")
print(f"Bentuk data setelah preprocessing: {X_preprocessed.shape}")

print(f"\n{'='*80}")
print("Nama fitur setelah preprocessing:")
print(f"{'='*80}")
for i, name in enumerate(feature_names_after, 1):
    print(f"{i:3d}. {name}")

# Konversi ke DataFrame untuk melihat beberapa baris pertama
X_preprocessed_df = pd.DataFrame(X_preprocessed, columns=feature_names_after)
print(f"\n{'='*80}")
print("5 baris pertama data setelah preprocessing:")
print(f"{'='*80}")
print(X_preprocessed_df.head())

print(f"\n{'='*80}")
print("Statistik deskriptif data setelah preprocessing:")
print(f"{'='*80}")
print(X_preprocessed_df.describe())

Jumlah fitur sebelum preprocessing: 44
Jumlah fitur setelah preprocessing: 107

Bentuk data asli: (11, 44)
Bentuk data setelah preprocessing: (11, 107)

Nama fitur setelah preprocessing:
  1. num__usia
  2. num__jumlah_anak
  3. num__usia_anak
  4. num__lama_bekerja
  5. num__waktu_bekerja_seminggu
  6. num__beban_sks
  7. num__mhs_bimbingan
  8. num__work_life_balance
  9. num__gaji_sesuai
 10. cat__jenis_kelamin_Laki-laki
 11. cat__jenis_kelamin_Perempuan
 12. cat__kota_asal_Bandar Lampung
 13. cat__kota_asal_Jembrana, Bali
 14. cat__kota_asal_Lampung
 15. cat__kota_asal_Lampung Selatan
 16. cat__kota_asal_Medan
 17. cat__kota_asal_Padangsidimpuan
 18. cat__kota_asal_Salatiga
 19. cat__kota_asal_Semarang
 20. cat__kota_asal_Yogyakarta 
 21. cat__status_pernikahan_Belum Menikah
 22. cat__status_pernikahan_Sudah Menikah
 23. cat__tinggal_sendiri_False
 24. cat__tinggal_sendiri_True
 25. cat__tinggal_pasangan_False
 26. cat__tinggal_pasangan_True
 27. cat__tinggal_anak_False
 28. cat__t

---

# Feature Selection

Feature selection is an important step in building machine learning models. It helps us:
1. **Reduce overfitting** by removing irrelevant or redundant features
2. **Improve model performance** by focusing on the most informative features
3. **Reduce training time** by decreasing the dimensionality of the dataset
4. **Enhance model interpretability** by identifying the key drivers

In this section, we will perform feature selection based on **correlation analysis**:
- **Correlation with target variable**: Identifies which features have the strongest relationship with the target
- **Multicollinearity detection**: Identifies highly correlated features that provide redundant information

In [16]:
# First, let's check the shape of our feature set
print(f"Original number of features: {X.shape[1]}")
print(f"Feature names: {X.columns.tolist()}")

# Separate numerical features for correlation analysis
# We'll only analyze numerical features as correlation works best with numeric data
X_numerical = X.select_dtypes(include=['int64', 'float64'])
print(f"\nNumerical features for correlation analysis: {X_numerical.shape[1]}")

Original number of features: 44
Feature names: ['usia', 'jenis_kelamin', 'kota_asal', 'status_pernikahan', 'jumlah_anak', 'usia_anak', 'tinggal_sendiri', 'tinggal_pasangan', 'tinggal_anak', 'tinggal_ortu', 'tinggal_mertua', 'tinggal_saudara', 'tinggal_teman', 'profesi', 'bidang', 'lama_bekerja', 'mode_bekerja', 'jarak', 'waktu_bekerja_seminggu', 'beban_sks', 'mhs_bimbingan', 'jabatan_struktural', 'jabatan_fungsional', 'sertifikasi', 'status_keaktifan', 'fisik_mata', 'fisik_punggung', 'fisik_tensi', 'fisik_lemah', 'fisik_kepala', 'fisik_obesitas', 'fisik_imun', 'fisik_carpal', 'mental_anxiety', 'mental_burnout', 'mental_depresi', 'mental_distress', 'mental_konsentrasi', 'mental_insomnia', 'mental_iritate', 'mental_lelah', 'mental_stres', 'work_life_balance', 'gaji_sesuai']

Numerical features for correlation analysis: 9


## 1. Correlation Matrix Visualization

Let's visualize the correlation matrix to understand the relationships between features and identify potential multicollinearity issues.

In [ ]:
# Calculate correlation matrix for numerical features
correlation_matrix = X_numerical.corr()

# Visualize the correlation matrix
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of Numerical Features', fontsize=16, pad=20)
plt.tight_layout()
plt.show()

## 2. Correlation with Target Variable

Now let's examine how each feature correlates with our target variable (`skor_total`). Features with higher absolute correlation values have a stronger linear relationship with the target.

In [ ]:
# Calculate correlation with target variable for numerical features only
target_correlation = X_numerical.corrwith(y).sort_values(ascending=False)

# Display correlation values
print("Correlation with Target Variable (skor_total):")
print("="*60)
for feature, corr in target_correlation.items():
    print(f"{feature:40s}: {corr:7.4f}")

# Visualize correlation with target
plt.figure(figsize=(10, 8))
target_correlation.plot(kind='barh', color='steelblue')
plt.title('Feature Correlation with Target Variable (skor_total)', fontsize=14, pad=15)
plt.xlabel('Correlation Coefficient', fontsize=12)
plt.ylabel('Features', fontsize=12)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 3. Detecting Multicollinearity

Multicollinearity occurs when two or more features are highly correlated with each other. This can cause problems in regression models because:
- It makes it difficult to determine the individual effect of each feature
- It can lead to unstable coefficient estimates
- It increases the variance of coefficient estimates

We'll identify pairs of features with high correlation (> 0.8 or < -0.8) and consider removing one from each pair.

In [ ]:
# Find highly correlated feature pairs
def find_high_correlation_pairs(corr_matrix, threshold=0.8):
    """
    Find pairs of features with correlation above threshold
    """
    high_corr_pairs = []
    
    # Get the upper triangle of the correlation matrix (to avoid duplicates)
    upper_triangle = np.triu(np.abs(corr_matrix), k=1)
    
    # Find pairs with high correlation
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            if upper_triangle[i, j] > threshold:
                high_corr_pairs.append({
                    'feature_1': corr_matrix.columns[i],
                    'feature_2': corr_matrix.columns[j],
                    'correlation': corr_matrix.iloc[i, j]
                })
    
    return pd.DataFrame(high_corr_pairs)

# Find highly correlated pairs (threshold = 0.8)
high_corr_df = find_high_correlation_pairs(correlation_matrix, threshold=0.8)

if len(high_corr_df) > 0:
    print(f"Found {len(high_corr_df)} pairs of highly correlated features (|r| > 0.8):")
    print("="*80)
    print(high_corr_df.to_string(index=False))
else:
    print("No highly correlated feature pairs found (threshold = 0.8)")

## 4. Feature Selection Strategy

Based on the correlation analysis, we'll select features using the following criteria:

1. **Remove one feature from highly correlated pairs** (to reduce multicollinearity)
   - Keep the feature with higher absolute correlation with the target
   
2. **Keep features with meaningful correlation with target** (optional threshold-based filtering)
   - You can set a minimum correlation threshold if needed

Let's implement this selection:

In [ ]:
# Function to select features based on correlation
def select_features_by_correlation(X, y, corr_threshold=0.8, min_target_corr=None):
    """
    Select features by:
    1. Removing one from each highly correlated pair (keeping the one with higher target correlation)
    2. Optionally filtering by minimum correlation with target
    
    Parameters:
    - X: feature dataframe
    - y: target variable
    - corr_threshold: threshold for detecting high correlation between features
    - min_target_corr: minimum absolute correlation with target (None = no filtering)
    """
    # Separate numerical and categorical features
    X_num = X.select_dtypes(include=['int64', 'float64'])
    X_cat = X.select_dtypes(include=['object', 'bool'])
    
    # Calculate correlations
    feature_corr_matrix = X_num.corr()
    target_corr = X_num.corrwith(y).abs()
    
    # Find features to remove due to multicollinearity
    features_to_remove = set()
    
    for i in range(len(feature_corr_matrix.columns)):
        for j in range(i+1, len(feature_corr_matrix.columns)):
            feat_i = feature_corr_matrix.columns[i]
            feat_j = feature_corr_matrix.columns[j]
            
            if abs(feature_corr_matrix.iloc[i, j]) > corr_threshold:
                # Remove the feature with lower correlation with target
                if target_corr[feat_i] < target_corr[feat_j]:
                    features_to_remove.add(feat_i)
                else:
                    features_to_remove.add(feat_j)
    
    # Apply minimum target correlation filter if specified
    if min_target_corr is not None:
        low_corr_features = target_corr[target_corr < min_target_corr].index.tolist()
        features_to_remove.update(low_corr_features)
    
    # Select features
    selected_numerical = [f for f in X_num.columns if f not in features_to_remove]
    
    # Combine with categorical features
    X_selected = pd.concat([X_num[selected_numerical], X_cat], axis=1)
    
    return X_selected, list(features_to_remove), selected_numerical

# Apply feature selection
# Adjust parameters as needed:
# - corr_threshold: how correlated features must be to consider removing one (default 0.8)
# - min_target_corr: minimum correlation with target (None = keep all, or set to e.g., 0.1)
X_selected, removed_features, selected_num_features = select_features_by_correlation(
    X, y, 
    corr_threshold=0.8, 
    min_target_corr=None  # Change to a value like 0.05 if you want to filter by target correlation
)

print(f"Original number of features: {X.shape[1]}")
print(f"Selected number of features: {X_selected.shape[1]}")
print(f"Number of features removed: {len(removed_features)}")
print(f"\nRemoved features:")
for feat in removed_features:
    print(f"  - {feat}")
print(f"\nSelected numerical features ({len(selected_num_features)}):")
for feat in selected_num_features:
    print(f"  - {feat}")

## 5. Update Features for Model Training

Now we'll update our feature set `X` to use the selected features. This will be used in the subsequent model training steps.

In [ ]:
# Update X with selected features
X = X_selected.copy()

print(f"✓ Feature set updated!")
print(f"  Total features: {X.shape[1]}")
print(f"  Samples: {X.shape[0]}")
print(f"\nFeature list:")
print(X.columns.tolist())

---

# Splitting Data Into 5 Folds

```python
kf = KFold(n_splits=5, shuffle=True, random_state=2024)
```

This line initializes a K-Fold cross-validator from scikit-learn's model selection module. Let's break down the parameters:

- `n_splits=5`: This sets up 5-fold cross-validation. The data will be divided into 5 equal parts or "folds".
- `shuffle=True`: This parameter tells the cross-validator to shuffle the data before splitting it into folds. This helps to ensure that the order of the data doesn't affect the results.
- `random_state=2024`: This sets a specific random seed for reproducibility. Using the same random state will ensure that the data is shuffled in the same way every time the code is run.

In 5-fold cross-validation:
1. The data is divided into 5 equal subsets or folds.
2. The model is trained on 4 folds and tested on the remaining fold.
3. This process is repeated 5 times, with each fold serving as the test set exactly once.
4. The performance metrics are then averaged across all 5 iterations.

This method helps to get a more robust estimate of the model's performance by using all the data for both training and testing.

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=2024)

---

# Define Model

In this experiment, I tried to simultaneously use five models: LinearRegression, XGBoost, RandomForest, GradientBoosting, and Support Vector Machine. The model that performs best will be used to make predictions on the test set.

In [ ]:
models = {
    'LR': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'XGB': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=2024))]),
    'RFR': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', RandomForestRegressor(n_estimators=100, random_state=2024))]),
    'GBR': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', GradientBoostingRegressor(n_estimators=100, random_state=2024))]),
    'SVR': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', SVR(kernel='rbf', C=1.0, epsilon=0.1))])
}

In [ ]:
results = {model: {'r2': [], 'rmse': [], 'mae': []} for model in models}

---

# Training

In this part of the code, we are training the machine learning model using the training data. The model learns the relationship between the input features and the target variable by adjusting its internal parameters based on the training data. The goal is to minimize the error between the predicted values and the actual values in the training set.

In [ ]:
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        results[name]['r2'].append(r2_score(y_test, y_pred))
        results[name]['rmse'].append(np.sqrt(mean_squared_error(y_test, y_pred)))
        results[name]['mae'].append(np.mean(np.abs(y_test - y_pred)))

---

# Evaluate Model

We use three metrics to evaluate the performance of the model:
* R2 Score: It is a statistical measure that represents the proportion of the variance for a dependent variable that's explained by an independent variable or variables in a regression model. The R2 score ranges from 0 to 1, where 1 indicates a perfect fit and 0 indicates no relationship between the independent and dependent variables.
* RMSE (Root Mean Squared Error): It is a measure of the differences between values predicted by a model and the values observed. It is the square root of the average of the squared differences between the predicted and actual values.
* MAE (Mean Absolute Error): It is a measure of errors between paired observations expressing the same phenomenon. It is the average of the absolute differences between the predicted and actual values.

In [ ]:
for name, metrics in results.items():
    print(f"{name} Results:")
    print(f"Average R-squared: {np.mean(metrics['r2']):.4f} (+/- {np.std(metrics['r2']):.4f})")
    print(f"Average RMSE: {np.mean(metrics['rmse']):.4f} (+/- {np.std(metrics['rmse']):.4f})")
    print(f"Average MAE: {np.mean(metrics['mae']):.4f} (+/- {np.std(metrics['mae']):.4f})")
    print()

Based on the results of the evaluation metrics, we can determine how well the model is performing and make adjustments as needed to improve its performance. From the five models, ..... performed the best, with the highest R2 score and lowest RMSE and MAE values.

---

# Check Feature Importance

We want to check, from all of the features, which features are the most important in predicting the target variable. This can help us understand which features have the most impact on the target variable and how they contribute to the model's predictions.

If the coefficients of the features are positive, it means that the feature has a positive impact on the target variable. If the coefficients are negative, it means that the feature has a negative impact (inverse) on the target variable.

We will sort the features based on their coefficients to identify the most important features in the model.

In [ ]:
def get_feature_importance(model, model_name, X, y):
    feature_names = model.named_steps['preprocessor'].get_feature_names_out()
    
    if model_name == 'LR':
        importances = model.named_steps['regressor'].coef_
    elif model_name in ['XGB', 'RFR', 'GBR']:
        importances = model.named_steps['regressor'].feature_importances_
    elif model_name == 'SVR':
        perm_importance = permutation_importance(model, X, y, n_repeats=10, random_state=2024)
        importances = perm_importance.importances_mean
    else:
        return None

    feature_importance = dict(zip(feature_names, importances))
    return dict(sorted(feature_importance.items(), key=lambda item: abs(item[1]), reverse=True))

# Visualizing Feature Importance

Understanding which features contribute most to a machine learning model’s predictions is crucial for interpretability and model refinement. In this section, we visualize **feature importance** for each model to see which variables have the greatest influence on the output (`skor_total`).

The following function, `plot_feature_importance`, retrieves, filters, and visualizes feature importances using a bar plot.

In [ ]:
def plot_feature_importance(model, model_name, X, y, threshold=0.01):
    importance = get_feature_importance(model, model_name, X, y)
    if importance is None:
        print(f"No feature importance available for {model_name}")
        return
    
    # Ubah ke DataFrame
    df_imp = pd.DataFrame(list(importance.items()), columns=['Feature', 'Importance'])
    
    # Filter fitur dengan nilai mendekati 0 (misal ambil yang absolut > threshold)
    df_imp = df_imp[df_imp['Importance'].abs() > threshold]

    if df_imp.empty:
        print(f"Semua feature importance di bawah threshold ({threshold}) untuk {model_name}")
        return

    # Urutkan dan plot
    df_imp = df_imp.sort_values(by='Importance', ascending=False)

    plt.figure(figsize=(8, max(4, len(df_imp) * 0.4)))
    sns.barplot(data=df_imp, x='Importance', y='Feature', palette='viridis')
    plt.title(f'Feature Importance ({model_name})')
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.tight_layout()
    plt.show()

# === Contoh Pemanggilan ===
for name, model in models.items():
    print(f"\nFeature Importance Plot for {name}:")
    plot_feature_importance(model, name, X, y, threshold=0.01)

## Saving the Importance Value as CSV

In [ ]:
def get_feature_importance(model, model_name, X, y):
    feature_names = model.named_steps['preprocessor'].get_feature_names_out()
    
    if model_name == 'LR':
        importances = model.named_steps['regressor'].coef_
    elif model_name in ['XGB', 'RFR', 'GBR']:
        importances = model.named_steps['regressor'].feature_importances_
    elif model_name == 'SVR':
        # For SVR, we use permutation importance
        perm_importance = permutation_importance(model, X, y, n_repeats=10, random_state=2024)
        importances = perm_importance.importances_mean
    else:
        return None

    return dict(zip(feature_names, importances))

# Calculate feature importances for all models
X = data[features]
y = data[target]

feature_importances = {}
for name, model in models.items():
    feature_importances[name] = get_feature_importance(model, name, X, y)

# Create a DataFrame from the feature importances
df_importance = pd.DataFrame(feature_importances)

# Sort the DataFrame by the average importance across all models
df_importance['avg_importance'] = df_importance.mean(axis=1)
df_importance = df_importance.sort_values('avg_importance', ascending=False)
df_importance = df_importance.drop('avg_importance', axis=1)

# Rename the index to include the categorical labels
new_index = []
for feature in df_importance.index:
    if feature.startswith('cat__'):
        parts = feature.split('__')
        if len(parts) == 3:
            new_index.append(f"{parts[1]}_{parts[2]}")
        else:
            new_index.append(feature)
    else:
        new_index.append(feature)
df_importance.index = new_index

# Save the DataFrame to a CSV file
df_importance.to_csv('feature_importance_comparison.csv')

print("Feature importance comparison has been saved to 'feature_importance_comparison.csv'")